<a href="https://colab.research.google.com/github/World4AI/World4AI/blob/main/website/src/notebooks/convolutional_neural_networks/vgg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VGG

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import torchvision.transforms as T
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt

import os
import pathlib
import zipfile
import shutil

In [2]:
!mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [3]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [4]:
!mv kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle competitions download -p /content/datasets  -c dogs-vs-cats

dogs-vs-cats.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
root = pathlib.Path('/content/datasets')

In [7]:
if not os.path.exists(root / 'dogs-vs-cats'):
  with zipfile.ZipFile(root / 'dogs-vs-cats.zip', 'r') as zip_ref:
      zip_ref.extractall(root / 'dogs-vs-cats')
      
  with zipfile.ZipFile(root / 'dogs-vs-cats/train.zip', 'r') as zip_ref:
      zip_ref.extractall(root / 'dogs-vs-cats')


In [8]:
original_path = root / 'dogs-vs-cats'
new_path = root / 'dogs_vs_cats_prepared'

In [9]:
# prepare list of files
file_names = os.listdir(original_path / 'train')
test_file_names = file_names[0:5000]
val_file_names = file_names[5000:10000]
train_file_names = file_names[10000:]

In [10]:
# prepare dirs
for directory in ["train", "test", "val"]:
    for category in ["cat", "dog"]:
        new_dir = new_path / directory / category
        if os.path.exists(new_dir):
            shutil.rmtree(new_dir)
        os.makedirs(new_dir)

In [11]:
def copy_files(old_dir, new_dir, files_list):
    for idx, filename in enumerate(files_list):
        src = old_dir / filename
        if filename.startswith('cat'):
            dst = new_dir / 'cat' / filename
        elif filename.startswith('dog'):
            dst = new_dir / 'dog' / filename
        else:
            continue
            
        shutil.copyfile(src=src, dst=dst)

In [12]:
copy_files(old_dir=original_path / "train", new_dir=new_path / 'train', files_list=train_file_names)
copy_files(old_dir=original_path / "train", new_dir=new_path / 'val', files_list=val_file_names)
copy_files(old_dir=original_path / "train", new_dir=new_path / 'test', files_list=test_file_names)

In [13]:
root = new_path

In [14]:
train_transform = T.Compose([T.Resize((256, 256)), 
                             T.RandomCrop(size=(224, 224)),
                             T.ToTensor()])

test_transform = T.Compose([T.Resize((224, 224)), 
                                T.ToTensor()])

In [15]:
train_dataset = ImageFolder(root=root / 'train', transform=train_transform)
val_dataset = ImageFolder(root=root / 'val', transform=test_transform)
test_dataset = ImageFolder(root=root / 'test', transform=test_transform)

In [16]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE=32

In [17]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=True)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, drop_last=False)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, drop_last=False)

In [18]:
cfg = [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"]

In [30]:
class Model(nn.Module):

    def __init__(self, cfg, num_classes=1):
        super().__init__()
        self.cfg = cfg
        self.feature_extractor = self._make_feature_extractor()
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(p=0.5),
            nn.Linear(512*7*7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 1)
        )
        
    def _make_feature_extractor(self):
        layers = []
        in_channels = 3
        for element in self.cfg:
            if element == "M":
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                conv2d = nn.Conv2d(in_channels, element, kernel_size=3, padding=1)
                layers += [conv2d, nn.BatchNorm2d(num_features=element), nn.ReLU(inplace=True)]
                in_channels = element
        return nn.Sequential(*layers)
        
    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.avgpool(x)
        x = self.classifier(x)
        return x

In [31]:
def track_performance(dataloader, model, criterion):
    # switch to evaluation mode
    model.eval()
    num_samples = 0
    num_correct = 0
    loss_sum = 0
    
    # no need to calculate gradients
    with torch.inference_mode():
        for batch_idx, (features, labels) in enumerate(dataloader):
            features = features.to(DEVICE)
            labels = labels.to(DEVICE).view(-1, 1).float()
            logits = model(features)
            probs = torch.sigmoid(logits)
                        
            predictions = (probs > 0.5).float()
            num_correct += (predictions == labels).sum().item()
            
            loss = criterion(logits, labels)
            loss_sum += loss.cpu().item()
            num_samples += len(features)
    
    # we return the average loss and the accuracy
    return loss_sum/num_samples, num_correct/num_samples

In [32]:
def train(num_epochs, train_dataloader, val_dataloader, model, criterion, optimizer, scheduler=None):
    history = {"train_loss": [], "val_loss": [], "train_acc": [], "val_acc": []}
    
    model.to(DEVICE)
    
    for epoch in range(num_epochs):
        for batch_idx, (features, labels) in enumerate(train_dataloader):
            model.train()
            features = features.to(DEVICE)
            labels = labels.to(DEVICE).view(-1, 1).float()
            
            # Empty the gradients
            optimizer.zero_grad()
            
            # Forward Pass
            logits = model(features)
            
            # Calculate Loss
            loss = criterion(logits, labels)
            
            # Backward Pass
            loss.backward()
            
            # Gradient Descent
            optimizer.step()
            
        train_loss, train_acc = track_performance(train_dataloader, model, criterion)
        val_loss, val_acc = track_performance(val_dataloader, model, criterion)

        if scheduler:
          scheduler.step(val_loss)

        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["train_acc"].append(train_acc)
        history["val_acc"].append(val_acc)

        print(f'Epoch: {epoch+1:>2}/{num_epochs} | Train Loss: {train_loss:.5f} | Val Loss: {val_loss:.5f} | Train Acc: {train_acc:.3f} | Val Acc: {val_acc:.3f}')
    return history            
            

In [33]:
model = Model(cfg)
optimizer = torch.optim.SGD(model.parameters(), momentum=0.9, lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                       factor=0.1,
                                                       mode='max',
                                                       patience=2,
                                                       verbose=True)
criterion = nn.BCEWithLogitsLoss()

In [35]:
history = train(20, train_dataloader, val_dataloader, model, criterion, optimizer, scheduler)

Epoch:  1/20 | Train Loss: 0.02167 | Val Loss: 0.02175 | Train Acc: 0.542 | Val Acc: 0.554
Epoch:  2/20 | Train Loss: 0.01941 | Val Loss: 0.01948 | Train Acc: 0.649 | Val Acc: 0.654
Epoch:  3/20 | Train Loss: 0.01968 | Val Loss: 0.02000 | Train Acc: 0.659 | Val Acc: 0.654
Epoch 00004: reducing learning rate of group 0 to 1.0000e-03.
Epoch:  4/20 | Train Loss: 0.01694 | Val Loss: 0.01687 | Train Acc: 0.745 | Val Acc: 0.737


KeyboardInterrupt: ignored

In [36]:
from torchvision.models import vgg19_bn, VGG19_BN_Weights

In [37]:
model = vgg19_bn(weights=VGG19_BN_Weights.IMAGENET1K_V1, progress=False)

Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /root/.cache/torch/hub/checkpoints/vgg19_bn-c79401a0.pth


In [38]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [39]:
for param in model.parameters():
    param.requires_grad = False

In [40]:
model.classifier[0].requires_grad = True
model.classifier[3].requires_grad = True

In [41]:
model.classifier[6] = nn.Linear(in_features=4096, out_features=1)

In [42]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [43]:
optimizer = optim.Adam(params=model.parameters(), lr=0.0001)
criterion = nn.BCEWithLogitsLoss()

In [44]:
history = train(10, train_dataloader, val_dataloader, model, criterion, optimizer)

Epoch:  1/10 | Train Loss: 0.00238 | Val Loss: 0.00230 | Train Acc: 0.984 | Val Acc: 0.983


KeyboardInterrupt: ignored